<a href="https://colab.research.google.com/github/LucianoTrujillo/7506R-2C2022-GRUPO10/blob/main/7506R_TP1_GRUPO10_ENTREGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#**TP1: Propiedades en Venta**



---


---


## *Facultad de Ingeniería, Universidad de Buenos Aires*

---
### *2do Cuatrimestre de 2022*

---
### *Organización de Datos*

### *Curso: Rodriguez*

### *Integrantes:*

* Luciano Leon Trujillo Palomo ltrujillo@fi.uba.ar 105664
* María Pilar Gaddi mgaddi@fi.uba.ar 105682
* Federico Adrian Solari Vazquez fsolariv@fi.uba.ar 106895
* Maria Vazquez Navarro mvazquezn@fi.uba.ar 105576
* Mateo Bulnes mbulnes@fi.uba.ar 106211
---

## **Objetivo del trabajo**

---
---

El objetivo de la siguiente investigación es aplicar técnicas de análisis exploratorio, preprocesamiento
de datos, agrupamiento, clasificación y regresión sobre un dataset real y poder obtener conclusiones y predicciones útiles. En especial, identificamos tres objetivos individuales:
* analizar si es posible agrupar los datos en función de
algún criterio, identificando a qué obedece el mismo.
* clasificar cada anuncio en tres categorías relacionadas al
precio de venta (alto, medio y bajo).
* predecir el precio de venta en dólares de una propiedad tipo
vivienda ubicada en Capital Federal.

---





## **Setup previo**


Importamos todas las librerías a utilizar a lo largo del trabajo:

In [773]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm

# para pca
from sklearn import datasets
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.preprocessing import StandardScaler



Importamos el dataset original y lo guardamos para su posterior uso:

In [ ]:

dwn_url='https://drive.google.com/uc?id=1z05Y6UwTu8fz1DpeE4JlRoyjJx_vPKmM'
ds_original = pd.read_csv(dwn_url)
ds_original.head(5)

In [ ]:
def mostrar_metadata_grafico(titulo, ylabel=None, xlabel=None, leyendas=None):
  plt.title(titulo)
  if ylabel:
    plt.ylabel = ylabel
  if xlabel:
    plt.xlabel = xlabel
  if leyendas:
    plt.legend = leyendas
  plt.show()



---
---


# Análisis Exploratorio y Preprocesamiento de Datos
---
---



## Pre-selección de datos
---
El primer paso consiste en la selección de los datos que se van a utilizar, se deben filtrar
únicamente los anuncios de propiedades de tipo vivienda ( Casa, PH y Departamento ) ubicados
en Capital Federal cuyo tipo de operación sea venta y su precio se encuentre en dólares (USD).

Primero filtramos por tipo de vivienda:

In [ ]:
ds = ds_original.copy()
ds = ds.query('property_type == "Casa" or property_type == "Departamento" or property_type == "PH"')

Luego filtramos los ubicados en Capital Federal: 



In [ ]:
ds.query('\
place_l3=="Capital Federal" or \
place_l4=="Capital Federal" or \
place_l5=="Capital Federal" or \
place_l6=="Capital Federal"').size

Observamos que la única columna que contiene a la clasificación de Capital Federal es place_l2, con lo cual con filtrar por l2, nos aseguramos de únicamente quedarnos con las filas que cumplan ser de Capital Federal.

In [ ]:
ds = ds.query('place_l2 == "Capital Federal"')

Filtramos por tipo de operación (Venta):

In [ ]:
ds = ds.query('operation == "Venta"')

Filtramos por moneda (USD):

In [ ]:
ds = ds.query('property_currency == "USD"')

Comparamos el tamaño del ds original y el que utilizaremos para el análisis en el trabajo:

In [ ]:
print("Tamaño original:", ds_original.shape, "Tamaño actual", ds.shape)

#### Filtrado de valores inválidos

#####Duplicados

Analizaremos la cantidad de duplicados del dataset.

In [ ]:
ds[ds.duplicated(subset=['id'])].shape[0]

Observamos que no hay datos duplicados segun el id. 

In [ ]:
columnas = ds.columns.to_list()#
columnas.remove('id')
ds_aux=ds[ds.duplicated(subset=columnas)]
ds_aux.shape[0]*100/ds.shape[0]


Con lo observado asumimos que el id se genera automaticamente al ingresar datos, sin antes verificar que los mismos no hayan sido ingresados antes.
Siendo que la proporcion de los datos duplicados resulta ser muy chica, nos parece correcto eliminarlos. 

Sin embargo notamos el hecho de que podrian presentarse casos de departamentos en los cuales todo sea igual, y se trate de dos departamentos en distintos pisos de un edificio, por lo tanto a continuacion analizaremos la frecuencia de esos casos.

In [ ]:
dup_casa_bool=ds_aux.property_type != "Departamento"
dup_casa_bool.sum()

ds_solo_deptos_duplicados=ds_aux.drop(ds_aux[dup_casa_bool].index) #dejamos esta variable, porque nos interesan las cantidad para anlaizar LO QUE FALTA
print(ds_aux.property_title.unique().size)
print(ds_solo_deptos_duplicados.property_title.unique().size)

Vemos que los datos de departamentos que se encuentran duplicados son 688, mientras que si eliminamos los duplicados, nos quedariamos con un total de 600 casos unicos. 

Nos parece razonable eliminar esos 88, siendo que en el peor de los casos solo estariamos perdiendo 88 departamentos que en realidad se encuentren en el mismo edificio, lo cual no nos parece que aporte tanto a nuestro modelo. 

Con los casos de PH y Casa no nos parece necesario hacer el analisis anteior, siendo que en estos casos la latitud y longitud si deberian cambiar en caso de tratarse de casas o PHs diferentes. Por lo tanto eliminamos los casos duplicados.

In [ ]:
print(ds.shape[0])
ds_a=ds.drop(ds[ds.duplicated(subset=columnas, keep='first')].index)
print(ds_a.shape[0])

##### tener mas superficie cubierta que superficie total

Notamos que los registros que tengan mas superficie cubierta que total seran invalidos, siendo que no se puede cubrir mas superficie que con la que se cuenta. Analizamos esos registros:

In [ ]:
print(ds[ds['property_surface_total'] < ds['property_surface_covered']].shape[0])
print(ds[ds['property_surface_total'] < ds['property_surface_covered']].shape[0]*100/ds.shape[0])

Observando bajas cantidades de estos registros decidimos eliminarlos del dataset.

In [ ]:
ds.drop(ds[ds['property_surface_total'] < ds['property_surface_covered']].index, inplace=True)

##### tener cubierta de 1.0 

In [ ]:
print(ds[ds['property_surface_covered'] == 1.0].property_surface_total)
print('porcentaje: ', ds[ds['property_surface_covered'] == 1.0].property_surface_total.shape[0]*100/ds.shape[0])



In [ ]:
ds.drop(ds[ds['property_surface_covered'] == 1.0].index , inplace=True)


Notamos que una cubierta de 1.0 no tiene sentido. Al analizar la total vemos que la mayoria so datos invalidos. Siendo que las cantidades de registros con estas caracteristicas es razonablemente pequeña a comparacion de la totalidad de los datos, decidimos eliminar los registros. 

##### tener mas habitaciones que ambientes

Realizamos un análisis sobre los posibles registros que podrían llegar a generar valores atípicos en  property_bedrooms. 

Debido a la alta correlación que tienen property_bedrooms y property_rooms, consideramos que property_rooms es el mejor candidato para determinar los valores atípicos de property_bedrooms. 

In [ ]:
pd.crosstab(ds.property_rooms, ds.property_bedrooms) 

Viendo la tabla, pudimos visulizar que en caso de que la cantidad de ambientes sea menor a la cantidad de habitaciones, property_bedrooms seria un valor atipico.

Se puede visualizar que una gran cantidad de registros de property_bedrooms son valores atípicos ya que los valores de los mismos son mayores a los valores de los registros de property_rooms. 

Además, notamos que hay valores de property_bedrooms negativos, con lo cual esos valores también serían atípicos.

In [ ]:
cond_mas_bedrooms_que_rooms = ds['property_bedrooms'] > ds['property_rooms']

reg_mas_bed_que_rooms = ds[cond_mas_bedrooms_que_rooms]
reg_mas_bed_que_rooms.shape

Como podemos observar hay 16604 registros de los cuales hay mas habitaciones en la propiedad que cantidad de ambientes, por lo tanto serian valores atipicos por lo que consideramos que hay que eliminarlos

In [ ]:
# sns.set(rc={"figure.figsize":(8, 8)}) #aumenta el tamaño del grafico
sns.scatterplot(data = ds, x = "property_rooms", y = "property_bedrooms")
sns.scatterplot(data = reg_mas_bed_que_rooms, x = "property_rooms", y = "property_bedrooms")#grafico de valores atipicos

In [ ]:
ds.drop(reg_mas_bed_que_rooms.index, inplace=True)# elimino las filas con mas bedrooms que rooms

In [ ]:
sns.scatterplot(data = ds, x = "property_rooms", y = "property_bedrooms")

In [ ]:
cond_bedrooms_menores_a_uno = ds['property_bedrooms'] < 1
reg_con_bedrooms_menores_a_uno = ds[cond_bedrooms_menores_a_uno]
reg_con_bedrooms_menores_a_uno.shape

Observamos que solo un registro contiene una cantidad de bedrooms menor o igual a cero, por ese motivo consideramos que debe ser eliminado

In [ ]:
ds.drop(reg_con_bedrooms_menores_a_uno.index, inplace=True)# elimino las filas con mas bedrooms menores a 1

In [ ]:
ds.columns.to_list()

A continuacion demostramos que no se encuentran valores negativos en las columnas en donde los valores siempre deben ser positivos, por lo tanto no es necesario eliminar datos.

In [ ]:
ds[ds['property_price'] < 1].shape 

In [ ]:
ds[ds['property_rooms'] < 1].shape

In [ ]:
ds[ds['property_surface_total'] < 1].shape

In [ ]:
ds[ds['property_surface_covered'] < 1].shape

##### created_on < start_date < end_date




In [ ]:
ds[ds['start_date'] < ds['created_on']].shape 

In [ ]:
ds[ds['end_date'] < ds['start_date']].shape 

In [ ]:
ds[ds['end_date'] < ds['created_on']].shape 

Como podemos observar todas las fechas de los avisos que se encuentran en el dataset cumplen con las condiciones necesarias para ser consideradas validas.


## Exploración Inicial
---


### Clasificación  de variables 


####Variables cuantitativas discretas

- start date, end date, created on: 

    Las fechas pueden ser consideradas cuantitativas discretas dado que toman valores numéricos ordenables, y entre dos valores consecutivos no existen valores intermedios. 

- property_rooms, property_bedrooms: 

    Pueden ser consideradas cuantitativas discretas ya que pueden tomar un conjunto a lo sumo numerable de valores, y cuentan cosas.

#### Variables cuantitativas continuas

- latitud y longitud:  

    Estas variables al tomar valores numéricos en los cuales existen infinitos valores intermedios (son arbitrariamente fraccionables), pueden ser consideradas como continuas. 

- property_surface_total, property_surface_covered, property_price: 

    Al igual que las variables anteriores, estas pueden tomar infinitos valores numéricos intermedios y por esa razón son consideradas continuas. 

####Variables cualitativas nominales

- id: 

    Es una variable cualitativa numérica que representa una identificación, cada una de estas es diferente y no tienen un orden visible. 

- place_l2, place_l3, place_l4, place_l5, place_l6: 

    Son variables de texto no trivialmente ordenables, dado que simplemente son un nombre correspondiente a la ubicación geográfica.

- operation, property_type, property_currency: 

    Son variables de texto que representan características para diferenciar propiedades entre sí, pero al igual que places, no tienen un ordenamiento predefinido. 

#### Variables irrelevantes


Observando las variables nombradas, consideramos las siguiente variables irrelevantes para el analisis:



- id: 

    El la campo Id es irrelevante ya que no aporta información sobre las viviendas, simplemente es utilizado para guardar la información en la base de datos.



- place_l5, place_l6:  
    
    Ambos campos son irrelevantes dado que no poseen datos que brinden información de gran importancia para el dominio del problema (todos los datos son nulos).



- property_title: 
    
    Dado que no haremos un análisis de NLP, no nos es relevante este campo.

###Variables cuantitativas
---


#### Cálculo medidas de resumen

Como debemos calcular la media, mediana, q1, q3 y moda para las variables start_date, end_date, created_on y las mismas son fechas, debemos convertirlas a un tipo de dato adecuado:

In [ ]:
ds_filtrado = ds.copy()
ds_filtrado['start_date']=pd.to_datetime(ds_filtrado['start_date'], errors='ignore')
ds_filtrado['created_on']=pd.to_datetime(ds_filtrado['created_on'], errors='ignore')

In [ ]:
try: 
    pd.to_datetime(ds_filtrado['end_date'])
except Exception as e: print(e)

Observamos que la columna end_date tiene una row cuyo valor es inválido: "9999-12-31", con lo cual averiguamos cuantos de éstos valores inválidos hay:

In [ ]:
invalid_end_dates = ds_filtrado.end_date[lambda date: pd.isnull(pd.to_datetime(date, errors='coerce'))]
invalid_end_dates.size


Encontramos 5041 apariciones de fecha invalidas. En la sección de Feature Engineering analizaremos que hacer con estos datos. <font color=red>Mientras tanto, hacemos un dataset que no contenga a las fechas inválidas para poder calcularle los valores pedidos.</font>

In [ ]:
ds_end_date = ds_filtrado.copy()
cond_operacion = pd.notnull(pd.to_datetime(ds_end_date['end_date'], errors='coerce'))
ds_end_date = ds_end_date[cond_operacion]
ds_end_date.end_date = pd.to_datetime(ds_end_date['end_date'])

In [ ]:
def mostrar_medidas_resumen(columna, ylabel=None, show_boxplot=True, df=ds_filtrado):
    mean=df[columna].mean()

    measure_info=np.nanpercentile(df[columna],[25,50,75])
    
    mode=df[columna].mode()

    print(f'Medidas de resumen para la variable {columna}:\n\
    \tMedia: {mean}\n\
    \tq1: {measure_info[0]}\n\
    \tMediana: {measure_info[1]}\n\
    \tq3: {measure_info[2]}\n\
    \tModa: {mode}')

    if(show_boxplot):
        sns.boxplot(y=df[columna], showfliers = False) # no mostramos outliers para ver mejor la distribución
        mostrar_metadata_grafico(f"distribución de {ylabel} en formato boxplot", ylabel=ylabel)

##### start date

In [ ]:
mostrar_medidas_resumen('start_date', show_boxplot=False)

#####created_on

In [ ]:
mostrar_medidas_resumen('created_on', show_boxplot=False)

#####end_date

In [ ]:
mostrar_medidas_resumen('end_date', show_boxplot=False, df=ds_end_date)

#####property_rooms

In [ ]:
mostrar_medidas_resumen('property_rooms', ylabel="cantidad de ambientes")

##### property_bedrooms

In [ ]:
mostrar_medidas_resumen('property_bedrooms', ylabel="cantidad de habitaciones")

#####latitud

In [ ]:
mostrar_medidas_resumen('latitud', ylabel="latitud (º)")

#####longitud

In [ ]:
mostrar_medidas_resumen('longitud', ylabel="longitud (º)")

#####property_surface_total

In [ ]:
mostrar_medidas_resumen('property_surface_total', ylabel="superficie total (m2)")

#####property_surface_covered

In [ ]:
mostrar_medidas_resumen('property_surface_covered', ylabel="superficie cubierta (m2)")

#####property_price

In [ ]:
mostrar_medidas_resumen('property_price', ylabel="precio (USD$)")

#### Análisis gráfico de distribuciones

In [ ]:
def mostrar_distribuciones_cuantitativas(variable, ds=ds_filtrado, figsize=(5,5), bins=None, binwidth=None, discrete=False, espaciado=4, rotacion=90):
  if binwidth:
    graph = sns.displot(ds, x=variable, discrete=discrete, bins=bins, binwidth=binwidth)
  else:
    graph = sns.displot(ds, x=variable, discrete=discrete, bins=bins)

  graph.fig.set_figheight(figsize[1])
  graph.fig.set_figwidth(figsize[0])
  ax = graph.ax
  mids = [rect.get_x() for rect in ax.patches]
  mids = [item for index, item in enumerate(mids) if index % espaciado == 0]
  ax.set_xticks(mids)
  ax.tick_params(axis='x', rotation=rotacion, labelsize=12)

  mostrar_metadata_grafico(f"distribución de {variable}", ylabel="frecuencia")



##### start_date

In [ ]:
mostrar_distribuciones_cuantitativas("start_date", figsize=(20, 8), bins=150)

##### end_date

In [ ]:
mostrar_distribuciones_cuantitativas("end_date", ds=ds_end_date, figsize=(20, 8), bins=150, espaciado=5)

#####created_on

In [ ]:
mostrar_distribuciones_cuantitativas("created_on", figsize=(20, 8), bins=150, espaciado=5)

#####property_rooms

In [ ]:
mostrar_distribuciones_cuantitativas("property_rooms", figsize=(20, 6), bins=[*range(20)], espaciado=1, rotacion=0)

#####property_bedrooms

In [ ]:
mostrar_distribuciones_cuantitativas("property_bedrooms", figsize=(20, 6), espaciado=1, bins=[*range(20)], rotacion=0)

#####property_surface_total

In [ ]:
ds_property_surface_total_sin_outliers = ds_filtrado[['property_surface_total']]
ds_property_surface_total_sin_outliers = ds_property_surface_total_sin_outliers[(ds_property_surface_total_sin_outliers.property_surface_total > 0) & (ds_property_surface_total_sin_outliers.property_surface_total < 1000)]
mostrar_distribuciones_cuantitativas("property_surface_total", ds=ds_property_surface_total_sin_outliers, figsize=(20, 6), bins=[*range(0, 1000, 10)], espaciado=2)

##### property_surface_covered
<font color='red'>REVISAR, tendría mas sentido que sea continuo el grafico </font>

In [ ]:
ds_property_surface_covered_sin_outliers = ds_filtrado[['property_surface_covered']]
ds_property_surface_covered_sin_outliers = ds_property_surface_covered_sin_outliers[(ds_property_surface_covered_sin_outliers.property_surface_covered > 0) & (ds_property_surface_covered_sin_outliers.property_surface_covered < 1000)]
mostrar_distribuciones_cuantitativas("property_surface_covered", ds=ds_property_surface_covered_sin_outliers, figsize=(20, 6), bins=[*range(0, 1000, 10)])

##### property_price
<font color='red'>REVISAR, tendría mas sentido que sea continuo el grafico </font>

In [ ]:
ds_property_price_sin_outliers = ds_filtrado[['property_price']]
ds_property_price_sin_outliers = ds_property_price_sin_outliers[(ds_property_price_sin_outliers.property_price > 0) & (ds_property_price_sin_outliers.property_price < 3000000)]
mostrar_distribuciones_cuantitativas("property_price", ds=ds_property_price_sin_outliers, figsize=(20, 6), bins=[*range(0, 3000000, 20000)], espaciado=4)


#####latitud

In [ ]:
ds_latitud_sin_outliers = ds_filtrado[['latitud']]
ds_latitud_sin_outliers = ds_latitud_sin_outliers[(ds_latitud_sin_outliers.latitud > -38) & (ds_latitud_sin_outliers.latitud < -34)]
mostrar_distribuciones_cuantitativas("latitud", ds=ds_latitud_sin_outliers, figsize=(20, 6), binwidth=0.005, espaciado=5)

#####longitud

In [ ]:
ds_longitud_sin_outliers = ds_filtrado[['longitud']]
ds_longitud_sin_outliers = ds_longitud_sin_outliers[(ds_longitud_sin_outliers.longitud > -59) & (ds_longitud_sin_outliers.longitud < -58)]
mostrar_distribuciones_cuantitativas("longitud", ds=ds_longitud_sin_outliers, espaciado=5, figsize=(20, 6), binwidth=0.005)

###### longitud y latitud en conjunto
<font color='red'>REVISAR, usar el random con una seed así siempre da el mismo resultado </font>

In [ ]:
import json
import shapely
import requests
import random
from shapely.geometry import shape, Point
from shapely.ops import unary_union
from shapely.ops import unary_union



url='https://drive.google.com/file/d/1eeT_53CFY63d55oZo50F7mOKPTZMW--V/view?usp=sharing'
file_id=url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id

url=requests.get(dwn_url)
data=url.text



data=json.loads(data)


caba_barrios = data
barrios = dict(
    (feature["properties"]["BARRIO"], shape(feature["geometry"])) 
    for feature in caba_barrios["features"]
)

all_barrios = unary_union(barrios.values())



def graficar_puntos_en_caba(longitudes, latitudes, tamano_de_punto=4,  proporcion=1):
  lat_long = list(zip(longitudes, latitudes))

  long_min, lat_min, long_max, lat_max = all_barrios.bounds

  viridis = cm.get_cmap('viridis', len(barrios) // 4)
  plasma = cm.get_cmap('plasma', len(barrios) // 4)
  inferno = cm.get_cmap('inferno', len(barrios) // 4)
  cividis = cm.get_cmap('cividis', len(barrios) - (len(barrios) // 4) * 3)
  colors = sum((list(x.colors) for x in (viridis, plasma, inferno, cividis)), [])

  random.shuffle(colors)


  f = plt.figure()
  f.set_figwidth(15)
  f.set_figheight(15)



  for idx, (barrio, shape) in enumerate(barrios.items()):
    for polygon in shape.geoms:
        plt.plot(*polygon.exterior.xy, color=colors[idx])
  
  values=lat_long.copy()
  random.shuffle(values)

  cont_outsiders=0
  cantidad = int(latitudes.size*proporcion)
  for dot in values[:cantidad]:     
    if(all_barrios.contains(Point(dot))):
       plt.scatter(x=dot[0], y=dot[1], s=tamano_de_punto, c='black')
    else: 
      cont_outsiders= cont_outsiders+1
 
  plt.show()
  print("El porcentaje de outsiders dentro de la proporcion pedida de la muestra es:", cont_outsiders/cantidad*100)

In [ ]:
graficar_puntos_en_caba(ds_filtrado.longitud, ds_filtrado.latitud, proporcion=0.005)

Como vemos, los puntos se distribuyen de manera coherente con la realidad, donde hay mas densidad de poblacion en sectores populares. Luego analizaremos los puntos faltantes y outsiders por separado, en la secciones correspondientes.   

### Variables Cualitativas
---


#### Cantidad de valores posibles y frecuencias

In [ ]:
def mostrar_frecuencias(columna, ancho=4, alto=4):
    frecuencia_por_valor = ds_filtrado.groupby(columna).size().to_frame("frecuencia")
    frecuencia_por_valor = frecuencia_por_valor.sort_values(by="frecuencia", ascending=False).reset_index()
    frecuencia_por_valor.head(40).plot.bar(x=columna, y="frecuencia", figsize=(ancho, alto))
    mostrar_metadata_grafico(f"frecuencias de valores posibles de {columna}", ylabel="frecuencia", xlabel="valores posibles")
    

#####place_l2

In [ ]:
mostrar_frecuencias('place_l2')

#####place_l3

In [ ]:
mostrar_frecuencias('place_l3', 10, 6)

#####place_l4

In [ ]:
mostrar_frecuencias('place_l4', 10, 5)

#####operation

In [ ]:
mostrar_frecuencias('operation', 10, 5)

#####property_type

In [ ]:
mostrar_frecuencias('property_type', 10, 5)

#####property_currency

In [ ]:
mostrar_frecuencias('property_currency', 10, 5)

## Análisis de Correlaciones existentes entre las variables
---



Eliminamos place_l6 al ser que tratan de elementos de tipo float64, siendo que no son validos como para evaluar correlaciones.

In [ ]:
matriz_temp = ds_filtrado.drop(columns=['place_l6'])
matriz_correlacion = matriz_temp.corr()

In [ ]:
plt.figure(figsize = (15,8))
sns.heatmap(matriz_correlacion, annot = True, cmap='Blues')
mostrar_metadata_grafico("Matriz de correlación entre los features del dataset")

Como observamos del heatmap, hay ciertas variables con una correlación significativa, por ejemplo:

- la cantidad de habitaciones vs la cantidad de ambientes
- superficie cubierta vs superficie total
- precio vs cantidad de ambientes y habitaciones 

Estas relaciones tienen sentido, dado que uno esperaría que a medida aumenta la cantidad de ambientes (y/o cuartos), la propiedad sea mas cara. También tiene mucho sentido que a mas superficie total, se puede tener mas superficie cubierta.

Lo que nos sorprende es que el precio de la propiedad parece tener una correlación muy débil con la superficie total y cubierta, lo cual nos asombra puesto que uno esperaría que a medida que aumentan los m^2 de la propiedad, aumenta su precio. 

## Separacion del dataset para train y test

#####<font color=red>HACER TODO EN TEST Y TRAIN 

A continuacion haremos la separacion de los los conjuntos de entrenamiento y test. 

Para respetar la idea de conservar proporciones utilizamos la herramienta de sklearn de stratify. A la misma debemos pasarle el target que nos interesa calcular en base a los datos del set. Por lo tanto a constinuacion clasificaremos dicho target.


###Clasificacion del Target

Buscamos un target cuyo tipo sea calificativo. Seguiremos la idea presentada por el enunciado de categorizar el precio como alto, bajo o medio. 

Para eso nos parece necesario analizar las medidas de resumen de property_price.

In [ ]:
mostrar_medidas_resumen('property_price', show_boxplot=True, df=ds_filtrado, ylabel='property price')

Observamos que el q1 es de aproximadamente 95000. Pensando en los casos reales que conocemos, nos parece un buen limite para poner el maximo de un precio que consideramos bajo. 

Luego, continuando con la distribucion de la variable, nos parece razonable clasificar un precio mediano con valores que ronden desde 95000 (el q1 de la variable) hasta 235000.

Por ultimo, consideramos que los valores superiores a 235000 ya se pueden clasificar como precios altos.

In [ ]:
def categorise(row):  
    if row['property_price'] <= 95000:
        return 'low_price'
    elif row['property_price'] > 95000 and row['property_price'] <= 235000:
        return 'medium_price'
    elif row['property_price'] > 235000:
        return 'high_price'


Una vez clasificados los tipos de precio en base a los precios, pasamos a crear la nueva columna.

In [ ]:
ds_filtrado['target'] = ds_filtrado.apply(lambda row: categorise(row), axis=1)


In [ ]:
ds_filtrado.target.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

#Creo un dataset con los features que voy a usar para clasificar
ds_filtrado_x=ds_filtrado.drop(['property_price'], axis='columns', inplace=False)

#Creo un dataset con la variable target
ds_filtrado_y = ds_filtrado['property_price'].copy()

#Genero los conjuntos de train y de test
x_train, x_test, y_train, y_test = train_test_split(ds_filtrado_x,
                                                    ds_filtrado_y,
                                                    stratify=ds_filtrado['target'].values, 
                                                    test_size=0.3,  #proporcion 70/30
                                                    random_state=2) #semilla

<font color=red>BALANCEAR

## Datos Faltantes
---


### Análisis de datos faltantes a nivel columna

In [ ]:
filas_totales = ds_filtrado.shape[0]
(ds_filtrado.isna().sum()/filas_totales*100).plot( kind = 'bar', 
             stacked = 'True',          # Muestra las barras apiladas
             alpha = 1,                 # nivel de transparencia
             width = 0.9,               # Grosor de las barras para dejar espacio entre ellas
             figsize=(9,4));            # Cambiamos el tamaño de la figura

Observamos que place l4 está conformado mayormente por valores vacíos, mientras que l5 y l6 están completamente vacíos. Luego tenemos varias columnas con todos sus valores, y algunas otras con un porcentaje de valores faltantes relativamente bajo.

### Análisis de datos faltantes a nivel fila

In [ ]:
temp = ds_filtrado.isna().sum(axis=1)
temp = temp.value_counts().to_frame("procentaje_sobre_cantidad_de_registros")
temp['valores_faltantes'] = temp.index
temp = temp.reset_index()
temp.procentaje_sobre_cantidad_de_registros = temp.procentaje_sobre_cantidad_de_registros/filas_totales*100
sns.barplot(data=temp, x="valores_faltantes", y="procentaje_sobre_cantidad_de_registros")
plt.show()


Podemos observar que contamos con una amplia cantidad de filas con 3 datos faltantes. Se observan pocas filas con mas de 6 datos faltantes, por lo que podemos asumir por ahora que la gran mayoría de nuestro data set nos brinda al menos mas del 50% de información en cada registro. 

### Reparaciones de Datos Faltantes


In [ ]:
(ds_filtrado.isna().sum()/filas_totales*100)

Si bien esta es el porcentaje de datos faltanes NaN, hay otros datos faltantes que son inválidos, y tambien los tenemos en cuenta.

Se observa que las columnas en las que se encuentran datos faltantes son en latitud, longitud, place_l3, place_l4, place_l5, place_l6, property_rooms, property_bedrooms, property_surface_total, y  property_surface_covered.
En esta seccion analizaremos como manejar estos casos especiales. Comenzaremos por la clasificacion de datos faltantes. 

* Missing completly at random:

    No existe relacion entre la falta de datos y la variable donde se encuentra el dato faltante.

* Missing not at random:

    La falta esta asociada a la naturaleza de la variable. Depende precisamente de los mismos datos que recolectamos. 


* Missing at random: 

    La falta no depende de los mismos datos faltantes, sino de otras variables. 


####Eliminacion de datos faltantes

#####place_l5 y place_l6

Se observa que place_l5 y place_l6 son datos faltantes de tipo MCAR ya que la razon de falta de datos es ajena a los datos mismos. Faltan datos en todos los registros, por lo que no se puede hacer ninguna relacion entre estos datos y otras columnas. 

In [ ]:
print(ds_filtrado['place_l5'].isna().sum()/filas_totales*100)
print(ds_filtrado['place_l6'].isna().sum()/filas_totales*100)

Dado que el porcentaje de datos faltantes es muestra una falta casi por completo de los datos, decimos eliminarlos.

In [ ]:
ds_filtrado.drop('place_l5', axis='columns', inplace=True)
ds_filtrado.drop('place_l6', axis='columns', inplace=True)

#####place_l4

In [ ]:
print("l4:\n", ds.place_l4.value_counts())
print("\nValores posibles de l4:\n", ds.place_l4.unique())

Se visualiza que place_l4 solo presenta datos sobre secciones dentro del barrio de Palermo. Si se enlistan los valores posibles en esta variable se aprecia el hecho de que los unicos valores son nan o secciones de Palermo. Por lo que se llega a la conclusion de que los datos faltantes podrian estar relacionados a que dichas celdas no presenten lugares de Palermo. En este caso, se abre la posibilidad de contar con que la variable place_l4 puede contar con datos del tipo MNAR (Missing not at random).  

In [ ]:
temp = ds_filtrado[ds_filtrado.place_l4.isna()]

plotdata = pd.DataFrame({
    "place l3 con l4":ds_filtrado.place_l3.value_counts(),
    "place l3 sin l4":temp.place_l3.value_counts(),
    }, 
)
plotdata.plot(kind="bar", figsize=(18,8))
plt.title("place_l3 con falta de place_l4 vs sin la falta de place_l4 ")
plt.xlabel = "Place_l3"


Se observa gráficamente que para que se observe un valor no nan en place_l4, place_l3 tiene que ser Palermo. Dado que vemos que aunque place_l3 sea palermo hay un gran procentaje donde falta place_l4, el hecho de que place_l3 sea Palermo es una condición necesaria pero no suficiente para observar place_l4 no nan.


In [ ]:
ds_filtrado[ds_filtrado['place_l4'].notna()].place_l3.value_counts()


Consideramos que place_l4 nos servia para verificar que la columna referente al barrio(place_l3) estuviera correcta, en este caso que fuera palermo en todas las filas que cuentan con un place_l4 con valor no nulo. 
Habiendo verificado que todas estan en palermo pordemos tomar la decision de que la columna no nos aporta informacion relevante para el dominio de nuestro problema, por lo tanto la eliminamos. 

In [ ]:
ds_filtrado.drop('place_l4', axis='columns', inplace=True)

##### latitud y longitud

Analizaremos estos dos datos juntos dado que tienen una relación lógica y tienen además el mismo procentaje de nans.

Primero analizamos si los registros que tienen nan en latitud son los mismos que tienen nan en longitud (y viceversa)

In [ ]:
cond_longitud_nan = ds_filtrado['longitud'].isna()
cond_latitud_nan = ds_filtrado['latitud'].isna()
print(ds_filtrado[cond_longitud_nan].shape)
print(ds_filtrado[cond_latitud_nan].shape)
print(ds_filtrado[cond_longitud_nan & cond_latitud_nan].shape)

Observamos que los registros que tienen alguno de los dos datos faltantes, también tiene su par faltante. Con lo cual, tiene sentido analizarlos juntos. Ademas consideramos apropiado clasificar estos datos faltantes, como MAR. Siendo que la falta de uno dependende del otro y viceversa.

Ahora analizaremos si los registros con estos datos faltantes corresponden a un barrio en específico, o si tienen alguna relación con la columna de barrio en general.

In [ ]:
registros_sin_coord = ds_filtrado[cond_longitud_nan & cond_latitud_nan]
registros_sin_coord.place_l3.value_counts()

Buscamos ahora cuantos registros no tienen ningún valor respecto a la ubicación:

In [ ]:
cond_barrio_nan = ds_filtrado['place_l3'].isna()
# cond_sector_barrial_nan = ds_filtrado['place_l4'].isna()
registros_sin_ubi = ds_filtrado[cond_longitud_nan & cond_latitud_nan & cond_barrio_nan ]#& cond_sector_barrial_nan]
registros_sin_ubi.shape

Tendría sentido remover estos registros dado que no hay variables con uan relación suficientemente fuerte para poder predecir la ubicación a partir de los otros datos. 

In [ ]:
ds_filtrado.drop(registros_sin_ubi.index, inplace=True)

A pesar de haber eliminado parte de los datos faltantes d eestas variables, continuaremos con el tratamiento de los restantes en la seccion de imputacion.

#### start_date, end_date, crated_on

Como vimos en la sección de exploración inicial -> variables cuantitativas -> medidas de resumen, teníamos 5041 registros inválidos de end_date dado que tenían una fecha "9999-12-31". 

Dado que esta fecha esta lejos de cualquier valor válido, no podemos aproximarla o transformarla en una fecha válida. 

#### <font color=red>Fechas

#### Imputacion de datos faltantes

#####property_bedrooms


In [ ]:
cond_habitaciones_nan = ds_filtrado['property_bedrooms'].isna()
cond_ambientes_nan = ds_filtrado['property_rooms'].isna()
registros_sin_rooms_ni_bedrooms = ds_filtrado[cond_habitaciones_nan & cond_ambientes_nan]
registros_sin_rooms_ni_bedrooms.shape

Consideramos que las filas que tienen cantidad de habitaciones y ambientes con valores nan habria que eliminarlas ya que no hay forma de obtener ambos datos con precision de una sin la otra debido a su alta correlacion.

In [ ]:
ds_filtrado.drop(registros_sin_rooms_ni_bedrooms.index, inplace=True)

In [ ]:
temp_sin_bedrooms = ds_filtrado[ds_filtrado.property_bedrooms.isna()]

plotdata = pd.DataFrame({
    "frecuencia de rooms":ds_filtrado.property_rooms.value_counts(),
    "frecuencia de rooms sin bedrooms":temp_sin_bedrooms.property_rooms.value_counts(),
    }, 
)

plotdata.plot(kind="bar", figsize=(18,8))
plt.title("property_bedrooms vs property_rooms sin property_bedrooms ")
plt.xlabel = "property_rooms"

Del grafico se puede analizar que en la vasta mayoria de los casos donde falta el dato sobre la cantidad de habitaciones es mayoritariamente en el caso de que se trate de monoambientes.

In [ ]:
temp_sin_bedrooms = ds_filtrado[ds_filtrado.property_bedrooms.isna()]

plotdata = pd.DataFrame({
    "sup_total con bedrooms":ds_filtrado.property_surface_total.value_counts(),
    "sup_total sin bedrooms":temp_sin_bedrooms.property_surface_total.value_counts(),
    }, 
)
plot_ = plotdata.plot(kind="bar", figsize=(18,8))
plt.title("property_surface_total con falta de property_bedrooms vs sin la falta de property_bedrooms ")
plt.xlabel = "property_surface_total"

for ind, label in enumerate(plot_.get_xticklabels()):
    if ind % 30 == 0:  # every 10th label is kept
        label.set_visible(True)
    else:
        label.set_visible(False)

En este grafico se puede observar que la mayoria de los registros en donde falta property_bedrooms es en las propiedades con superficie mas chica, fortaleciendo asi la hipótesis de que se encuentra una falta de datos en la columna de cantidad de habitaciones al tratarse de monoambientes. 


In [ ]:
ds_filtrado[(ds_filtrado['property_rooms']==1)].property_bedrooms.value_counts()


<font color='red'>los valores atípicos serán tratados en su respectiva sección</font> 


Dado que tener más de 1 habitación en un monoambiente es incoherente, y que la vasta mayoría de los monoambientes que no tienen dato faltante en propert_bedrooms tienen su valor en 1, reemplazamos lo registros monoambientes faltantes con 1 (asumiendo el supuesto de que las propiedades analizadas son hogares). 

Para los casos donde no son monoambientes, cuando predigamos la cantidad de habitaciones, reemplazaremos los 0's por 1's de igual manera mediante el mismo razonamiento. 

In [ ]:
columnas = ['property_rooms','property_bedrooms']
ds_property_bedrooms_sin_nan = ds_filtrado[columnas].copy()
cond_monoambiente = ds_property_bedrooms_sin_nan['property_rooms']==1
cond_bedroom_nan = ds_property_bedrooms_sin_nan['property_bedrooms'].isna()
ds_property_bedrooms_sin_nan.loc[cond_monoambiente & cond_bedroom_nan, 'property_bedrooms'] = 1 # reemplazamos monoambientes con bedrooms nan por 1


In [ ]:
print('El porcentaje de viviendas con mas de un ambiente (property_rooms) que no tienen habiaciones (property_bedrooms) es:', ds_filtrado['property_bedrooms'].isna().sum()/filas_totales*100)

Por lo tanto, la mayoría de los casos en los cuales faltan habitaciones (property_bedrooms) es en monoambientes. Con lo cual, podemos asumir que los datos faltantes de esta variable son de tipo MAR. 

Para los casos donde se tiene mas de un ambiente, podemos aplicar el imputador iterativo. 

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

columnas=['property_surface_total','property_surface_covered','property_bedrooms','property_rooms']
df_eliminar_nans_en_bedrooms_y_rooms = ds_filtrado[columnas].copy()

lr = LinearRegression()
imp = IterativeImputer(estimator=lr,missing_values=np.nan, max_iter=20, verbose=0, random_state=2) #imputation_order='roman'
array_sin_nans_en_bedrooms_y_rooms = imp.fit_transform(df_eliminar_nans_en_bedrooms_y_rooms)

In [ ]:
ds_sin_nans_en_bedrooms_y_rooms = pd.DataFrame(array_sin_nans_en_bedrooms_y_rooms, columns = columnas) #imp.fit_transfor devuelve un arrar sin nans en las columnas
ds_sin_nans_en_bedrooms_y_rooms = ds_sin_nans_en_bedrooms_y_rooms.round(0)

In [ ]:
plotdata = pd.DataFrame({
    "frecuencia de bedrooms con manejo de valores faltantes en general":ds_sin_nans_en_bedrooms_y_rooms.property_bedrooms.value_counts(),
    "frecuencia de bedrooms con manejo de valores faltantes en monoambiente ":ds_property_bedrooms_sin_nan.property_bedrooms.value_counts(),
    "frecuencia de bedrooms antes de manejo de valores faltantes":ds_filtrado.property_bedrooms.value_counts(),
    }, 
)

plotdata.plot(kind="bar", figsize=(18,8))
plt.title("frecuencias de bedrooms pre reemplazo vs post reemplazo de nans")
plt.xlabel = "property_bedrooms"

In [ ]:
print("Porcentaje de valores nan en property_bedrooms antes de manejar los valores faltantes:", ds_filtrado['property_bedrooms'].isna().sum()/filas_totales*100)

ds_filtrado['property_bedrooms'] = ds_property_bedrooms_sin_nan['property_bedrooms']
ds_filtrado.loc[:,"property_bedrooms"].replace(to_replace=[ds_filtrado['property_bedrooms']],value=[ds_sin_nans_en_bedrooms_y_rooms['property_bedrooms']],inplace=True)

print("Porcentaje de valores nan en property_bedrooms despues de manejar los valores faltantes:", ds_filtrado['property_bedrooms'].isna().sum()/filas_totales*100)

##### Latitud y Longitud

In [ ]:
cond_longitud_nan = ds_filtrado['longitud'].isna()
cond_latitud_nan = ds_filtrado['latitud'].isna()
print(ds_filtrado[cond_longitud_nan & cond_latitud_nan].shape)



Recordamos la cantidad de datos faltantes restantes que hay en estas dos variables, a continuacion efectuamos la imputacion de los datos.

Debido a que latitud y longitud son variables utilizadas para representar una ubicacion en especifico, pensamos en utilizar place_l3 y <font color='red'>place_l4</font> para predecir los valores faltantes siendo que estas tambien representan ubicaciones. Al ser ambas variables categoricas no es posible utilizarlas en <font color='red'>metodos que utilicen regresion lineal. </font> 
Por este motivo, decidimos solo utilizar para predecir el valor de las mismas a property_surface_total y property_price dado que el m^2 en una determinada ubicacion varia con respecto a el de otras. 


A continuacion vamos a hacer el proceso de imputacion de los datos faltantes de latitud y longitud, pero no lo haremos con las dos variables al simultaneo ya que al hacerlo por separado podemos obtener mejores resultados.

In [ ]:
ds_a_imputar= ds_filtrado.copy()

In [ ]:
columnas=['latitud', 'property_price', 'property_surface_total']
df_eliminar_nans_en_latitud = ds_a_imputar[columnas].copy()

imp = IterativeImputer(missing_values=np.nan, max_iter=20, verbose=0, random_state=2) #imputation_order='roman'
array_sin_nans_en_latitud = imp.fit_transform(df_eliminar_nans_en_latitud)

In [ ]:
ds_sin_nans_en_latitud = pd.DataFrame(array_sin_nans_en_latitud, columns = columnas) #imp.fit_transfor devuelve un arrar sin nans en las columnas
ds_a_imputar.loc[:,"latitud"].replace(to_replace=[ds_a_imputar['latitud']],value=[ds_sin_nans_en_latitud['latitud']],inplace=True)

In [ ]:
columnas=['latitud','longitud', 'property_price', 'property_surface_total']
df_eliminar_nans_en_latitud_y_longitud = ds_a_imputar[columnas].copy()

array_sin_nans_en_latitud_y_longitud = imp.fit_transform(df_eliminar_nans_en_latitud_y_longitud)

In [ ]:
ds_sin_nans_en_latitud_y_longitud = pd.DataFrame(array_sin_nans_en_latitud_y_longitud, columns = columnas) #imp.fit_transfor devuelve un arrar sin nans en las columnas
ds_a_imputar.loc[:,"longitud"].replace(to_replace=[ds_a_imputar['longitud']],value=[ds_sin_nans_en_latitud_y_longitud['longitud']],inplace=True)

In [ ]:
#accedemos a los indices de las filas que tienen nans
filas=ds_a_imputar[ds_filtrado['longitud'].isna()].index

graficar_puntos_en_caba(ds_sin_nans_en_latitud_y_longitud.longitud, ds_sin_nans_en_latitud_y_longitud.latitud, proporcion=0.005)

In [ ]:
#ACA HAY QUE VER SI LAS COORDENADAS QUE PREDIJO EL ALGORITMO PERTENECEN AL PLACE L3 DE LA FILA CORRESPONDIENTE

<font color=red>LUEGO COMPLETAR EL DS_FILTRADO</font>

#####place_l3

In [ ]:
print("cantidad de registros place_l3 nulos:", ds_filtrado.place_l3.isna().sum())

In [ ]:
cond_barrio_nan = ds_filtrado['place_l3'].isna()
# cond_sector_barrial_nan = ds_filtrado['place_l4'].isna()
registros_sin_ubi = ds_filtrado[cond_barrio_nan]
registros_sin_ubi.shape

In [ ]:
graficar_puntos_en_caba(registros_sin_ubi.longitud, registros_sin_ubi.latitud, proporcion=1)

Observando las latitudes y longitudes de los registros de los cuales place_l3 sus valores son nan, notamos que sus respectivas coordenadas se encuentran fuera de los limites de Capital Federal. Acontinuacion analizaremos como manejar estas incongruencias.

<font color='red'>CREEMOS QUE EL 80% QUE NO ESTA EN CABA HAY QUE ELIINARLO PERO TENEMOS QUE JUSTIFICAR BIEN. EL 20% QUE ESTA EN CABA HAY QUE, EN BASE A LA LAT Y LONG DETERMINAR EL PLACE_L3 MEDIANTE EL GRAFIQUITO. LUCIANO DIBUJE MAESTRO.</font> 

#####property_rooms

In [ ]:
print("Cantidad de valores faltantes de property_rooms:",ds_filtrado['property_rooms'].isna().sum())

Consideramos que podemos utilizar un metodo de regresion lineal para llenar los datos nan de property_rooms. Debido a la fuerte correlacion observada en la matriz de correlaciones, entre esta variable con property_bedrooms, decidimos utilizar esa columna para el entrenamiento de este modelo.


In [ ]:
# Guardamos un auxiliar de las dos columnas a tratar
ds_aux = ds_filtrado[['property_rooms', 'property_bedrooms']].copy()
ds_aux = ds_aux.query('property_rooms  > 0')
ds_aux.shape

In [ ]:
modelo_lineal_rooms = LinearRegression()

#Doy el formato adecuado a las variables dependiente e independiente
bedrooms=ds_aux['property_bedrooms'].values.reshape(-1, 1)
rooms=ds_aux['property_rooms'].values.reshape(-1, 1)

#Ajusto el modelo
modelo_lineal_rooms.fit(bedrooms, rooms)

#Coeficientes Estimados
intercepcion=round(modelo_lineal_rooms.intercept_[0],2)
coef_bedroom=round(modelo_lineal_rooms.coef_[0][0],2)

#Muestro el modelo
print(f" El modelo lineal simple ajustado es: rooms = {coef_bedroom} * bedrooms  + {intercepcion}")

In [ ]:
rooms_estim_segun_bedrooms= coef_bedroom * bedrooms + intercepcion

ds_aux['rooms_estimadas_bedrooms']= coef_bedroom * bedrooms + intercepcion #Lo agregamos en el dataframe
ds_aux.head()

Siendo que las variables de room son de tipo entera, debemos redondear los resultados.

In [ ]:
ds_aux = ds_aux.round(0) #los resultados deben ser numeros enteros

In [ ]:
plt.scatter(ds_filtrado['property_bedrooms'],ds_filtrado['property_rooms'],color='blue',alpha=0.5)
plt.scatter(ds_aux['property_bedrooms'],ds_aux['rooms_estimadas_bedrooms'], color='red',alpha=0.5)

In [ ]:
ds_antes_de_imputar = ds_filtrado.copy()

In [ ]:
#ahora calculo los valores de las rooms en base a las bedrooms
bedrooms_con_rooms_nan = ds_filtrado[ds_filtrado['property_rooms'].isna()]['property_bedrooms']
bedrooms_con_rooms_nan = bedrooms_con_rooms_nan.values.reshape(-1, 1)

rooms_estimadas_segun_bedrooms = modelo_lineal_rooms.predict(bedrooms_con_rooms_nan) #prediccion de valores
rooms_estimadas_segun_bedrooms = rooms_estimadas_segun_bedrooms.round(0)

ds_antes_de_imputar.loc[ds_antes_de_imputar['property_rooms'].isnull(), 'property_rooms'] = rooms_estimadas_segun_bedrooms

In [ ]:
plotdata = pd.DataFrame({
    "frecuencia de rooms con manejo de valores faltantes en general":ds_antes_de_imputar['property_rooms'].value_counts(),
    "frecuencia de rooms antes de manejo de valores faltantes":ds_filtrado['property_rooms'].value_counts(),
    }, 
)

plotdata.plot(kind="bar", figsize=(18,8))
plt.title("frecuencias de rooms pre reemplazo vs post reemplazo de nans")
plt.xlabel = "property_rooms"

In [ ]:
# remplazo de columna property_rooms
ds_filtrado.loc[:,"property_rooms"].replace(to_replace=[ds_filtrado['property_rooms']],value=[ds_antes_de_imputar['property_rooms']],inplace=True)

#####property_surface_total

Dado que property_surface_total y property_surfae_covered tienen una alta correlacion, analizamos los casos en los cuales ambas variables tiene valores nan.

In [ ]:
cond_superficie_total_nan = ds_filtrado['property_surface_total'].isna()
cond_superfice_cubierta_nan = ds_filtrado['property_surface_covered'].isna()
registros_sin_superficie_cubierta_ni_total = ds_filtrado[cond_superficie_total_nan & cond_superfice_cubierta_nan]
registros_sin_superficie_cubierta_ni_total.shape

Como ningun registro tiene ambas variables con valor en nan, no eliminamos ningun registro. 

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

columnas=['property_surface_total','property_surface_covered','property_rooms']
df_eliminar_nans_en_sup_total_y_cubierta = ds_filtrado[columnas].copy()

lr = LinearRegression()
imp = IterativeImputer(estimator=lr,missing_values=np.nan, max_iter=20, verbose=0, random_state=2) #imputation_order='roman'
array_sin_nans_en_sup_total_y_cubierta = imp.fit_transform(df_eliminar_nans_en_sup_total_y_cubierta)

In [ ]:
ds_sin_nans_en_sup_total_y_cubierta = pd.DataFrame(array_sin_nans_en_sup_total_y_cubierta, columns = columnas) #imp.fit_transfor devuelve un arrar sin nans en las columnas

In [ ]:
ds_sin_nans_acotado = ds_sin_nans_en_sup_total_y_cubierta.query('property_surface_total < 250')
ds_filtrado_acotado = ds_filtrado.query('property_surface_total < 250')

plotdata = pd.DataFrame({
    "frecuencia despues de superficie cubierta":ds_sin_nans_acotado.property_surface_total,
   "frecuencia antes de superficie cubierta":ds_filtrado_acotado.property_surface_total,
    }, 
)

plt.figure()
plotdata.plot.hist(alpha=0.5, bins = 100)

Decidimos acotar los valores de los registros property_surface_total dado que observamos que con valores mayores a 250 la cantidad de regitros era minima y no se apreciaba completamente el grafico.  

In [ ]:
print("Porcentaje de valores nan en property_surface_total antes de manejar los valores faltantes:", ds_filtrado['property_surface_total'].isna().sum()/filas_totales*100)

ds_filtrado.loc[:,"property_surface_total"].replace(to_replace=[ds_filtrado['property_surface_total']],value=[ds_sin_nans_en_sup_total_y_cubierta['property_surface_total']],inplace=True)

print("Porcentaje de valores nan en property_surface_total despues de manejar los valores faltantes:", ds_filtrado['property_surface_total'].isna().sum()/filas_totales*100)

#####property_surface_covered

Como property_surface_covered esta fuertemente correlacionado con property_surface_total utilizamos los mismos datos obtenidos en el manejo de valores faltantes de property_surface_total para imputar los datos faltantes de porperty_surface_covered. Para ello utilizamos el dataset generado anteriormente en la seccion property_surface_total.

In [ ]:
ds_sin_nans_acotado = ds_sin_nans_en_sup_total_y_cubierta.query('property_surface_covered < 500')
ds_filtrado_acotado = ds_filtrado.query('property_surface_covered < 500')

plotdata = pd.DataFrame({
    "frecuencia despues de superficie cubierta":ds_sin_nans_acotado.property_surface_covered,
   "frecuencia antes de superficie cubierta":ds_filtrado_acotado.property_surface_covered,
    }, 
)

plt.figure()
plotdata.plot.hist(alpha=0.5, bins = 100)

#Ver grafico, sacamos el value_counts en pd.DataFrame

In [ ]:
print("Porcentaje de valores nan en property_surface_covered antes de manejar los valores faltantes:", ds_filtrado['property_surface_covered'].isna().sum()/filas_totales*100)

ds_filtrado.loc[:,"property_surface_covered"].replace(to_replace=[ds_filtrado['property_surface_covered']],value=[ds_sin_nans_en_sup_total_y_cubierta['property_surface_covered']],inplace=True)

print("Porcentaje de valores nan en property_surface_covered despues de manejar los valores faltantes:", ds_filtrado['property_surface_covered'].isna().sum()/filas_totales*100)

##Valores Atípicos
---


### Análisis univariados


In [ ]:
def mostrar_atipicos_univariados(columna):
  sns.boxplot(y=columna)
  #Cuartiles
  Q1_altura=np.quantile(columna, 0.25)
  Q3_altura=np.quantile(columna, 0.75)

  #Rango intercuartil
  IQR_altura=Q3_altura-Q1_altura

  #calculo outliers moderados:
  out_inf=Q1_altura-1.5*IQR_altura
  out_sup=Q3_altura+1.5*IQR_altura

  print("limite moderado inferior:", out_inf)
  print("limite moderado superior:", out_sup)


  print("outliers por debajo de limite moderado", columna[(columna < out_inf)].size)
  print("outliers por arriba de limite moderado", columna[(columna > out_sup)].size)


  #Limite inferior Outliers
  out_inf=Q1_altura-3*IQR_altura
  out_sup=Q3_altura+3*IQR_altura

  print("limite severo inferior:", out_inf)
  print("limite severo superior:", out_sup)

  print("outliers por debajo de limite severo", columna[(columna < out_inf)].size)
  print("outliers por arriba de limite severo", columna[(columna > out_sup)].size)

  


####property_price

In [ ]:
mostrar_atipicos_univariados(ds_filtrado.property_price)

Estos resultados tienen sentido dado que no vemos ningún precio por debajo de 0, mientras que podemos observar precios de hogares que se son mucho mas altos que los límites brindados por el método IRQ.

####property_surface_total

In [ ]:
mostrar_atipicos_univariados(ds_filtrado.property_surface_total)

Estos resultados tienen sentido dado que no vemos ninguna medida de superficie por debajo de 0, mientras que podemos observar superficies que son mucho mas altos que los límites brindados por el método IRQ.

####property_surface_covered

In [ ]:
mostrar_atipicos_univariados(ds_filtrado.property_surface_covered)

idem anteriores


####property_bedrooms

In [ ]:
mostrar_atipicos_univariados(ds_filtrado.property_bedrooms)

####property_rooms

In [ ]:
mostrar_atipicos_univariados(ds_filtrado.property_rooms)

Como podemos observar, en la mayoría de los datos tenemos un rango bastante acotado de valores comunes, y los outliers se alejan en gran medida de ese rango acotado. Por eso es que en los gráficos de boxplot, con suerte se llega a ver el rango común.

### Análisis bivariados


In [ ]:
from sklearn.utils.fixes import scipy
def mostrar_atipicos_bivariados(columna_a, columna_b):
  sns.scatterplot(x=columna_a, y=columna_b)

# el umbral es a mano, viendo lo que mas sentido tenga
def analisis_mahalanobis(columnas, umbral, cantGraficos = 2):

  X = ds_filtrado[columnas]
  sample_X = X.sample(frac=0.1, random_state=2)

  mu = sample_X.mean()
  X_diff = X - mu
  X_diff = X_diff.values.T

  cov = np.cov(sample_X.values.T)
  inv_cov = np.linalg.inv(cov)
  W = scipy.linalg.sqrtm(inv_cov)

  Wx = np.matmul(W, X_diff)

  mahal_dists = np.array([np.dot(Wx[:, i], Wx[:, i]) for i in range(Wx.shape[1])])
 
  ds_mahal = ds_filtrado[columnas].copy()
  ds_mahal['mahalanobis'] = mahal_dists

  np.sort(ds_mahal)

  ds_mahal['es_outlier'] = ds_mahal['mahalanobis']>umbral

  if(cantGraficos > 1):
    ds_temp = ds_mahal.copy()
    distancia_menor_a_10 = ds_temp['mahalanobis'] < 15
    ds_temp = ds_temp[distancia_menor_a_10]
    print(ds_temp['es_outlier'].value_counts())
    mostrar_distribuciones_cuantitativas("mahalanobis", ds=ds_temp, figsize=(8, 8), bins=30, espaciado=2)
    if len(columnas) == 2:
      plt.figure(figsize=(8, 8), dpi=80)
      sns.scatterplot(x=ds_mahal[columnas[0]], y=ds_mahal[columnas[1]], hue=ds_mahal['es_outlier'])
      mostrar_metadata_grafico(f'Dispersograma {columnas[0]} vs {columnas[1]}')
  else:
    plt.figure(figsize=(8, 8), dpi=80)
    sns.scatterplot(x=ds_mahal[columnas[0]], y=ds_mahal[columnas[1]], hue=ds_mahal['es_outlier'])
    mostrar_metadata_grafico(f'Dispersograma {columnas[0]} vs {columnas[1]}')

####start_date vs end_date

In [ ]:
#VER ESTO #mostrar_atipicos_bivariados(ds_filtrado.start_date, ds_filtrado.end_date)

#### property_rooms vs property_bedrooms

In [ ]:
mostrar_atipicos_bivariados(ds_filtrado.property_rooms, ds_filtrado.property_bedrooms)

In [ ]:
analisis_mahalanobis(['property_rooms', 'property_bedrooms'], 7)

Como podemos observar del grafico de distribución de distancias mahalanobis, vemos que un umbral razonable es 7, dado que a partir de ahí las distancias grandes son anomalías. 
También podemos observar que de los 92000 registros, 2000 salieron como outliers, lo cual es bastante razonable. En el gráfico no parece que sean pocos, pero esto es debido a que la mayoría de los no outliers, se concentran en la esquina inferior izquierda, llevando el valor medio a esa área.

En este caso, proponemos eliminar aquellos outliers que cuentan con una amplia cantidad de ambientes y cuya cantidad de cuartos no es acorde lógicamente a ese número. 
Visualizando el scatterplot, tomamos como umbral de property_bedrooms <= cte * property_rooms siendo cte = 8/17 

In [ ]:
cond_menos_bedrooms_que_rooms = ds_filtrado['property_bedrooms'] <= 8/12*(ds_filtrado['property_rooms']) - 10/3

reg_menos_bed_que_rooms = ds_filtrado[cond_menos_bedrooms_que_rooms]
reg_menos_bed_que_rooms.shape

ds_filtrado.drop(reg_menos_bed_que_rooms.index, inplace=True)

In [ ]:
analisis_mahalanobis(['property_rooms', 'property_bedrooms'], 7, 1)

####property_surface_total vs property_surface_covered


In [ ]:
mostrar_atipicos_bivariados(ds_filtrado.property_surface_total, ds_filtrado.property_surface_covered)

In [ ]:
analisis_mahalanobis(['property_surface_total', 'property_surface_covered'], 1)

En este caso, vemos del grafico de distribución de distancias mahalanobis, que un umbral razonable seria 1, dado que a partir de ahí las distancias grandes comienzan a ser anomalías. 

Como analisis logico nos parece razonable destacar el hecho de que es muy poco probable tener Casa, PH o Departamentos que cuenten con mas de 2000 m2 en superficies cubiertas o totales en Capital Federal. Decidimos analizar cuantos casos tienen estas caracteristicas.


In [ ]:
cant = ds_filtrado[(ds_filtrado['property_surface_covered'] > 2000) | (ds_filtrado['property_surface_total'] > 2000)].shape[0]
total = ds_filtrado.shape[0]
cant*100/total


Vemos que la proporcion de datos con esas caracteristicas es muy poca a comparacion con la totalidad de los registros. Decidimos eliminar los datos al considerarlos logicamente invalidos. 

In [ ]:
cond_surface_total = ds_filtrado['property_surface_total'] > 2000
cond_surface_covered = ds_filtrado['property_surface_covered'] > 2000

reg_combinacion = ds_filtrado[cond_surface_total | cond_surface_covered]
reg_combinacion.shape

ds_filtrado.drop(reg_combinacion.index, inplace=True)

In [ ]:
mostrar_atipicos_bivariados(ds_filtrado.property_surface_total, ds_filtrado.property_surface_covered)

In [ ]:
cond_mucho_total_poco_covered = ds_filtrado['property_surface_covered'] <= 13/15*(ds_filtrado['property_surface_total']) -2860/3

reg_mucho_total_poco_covered = ds_filtrado[cond_mucho_total_poco_covered]
reg_mucho_total_poco_covered.shape

ds_aux=ds_filtrado.drop(reg_mucho_total_poco_covered.index, inplace=False)
mostrar_atipicos_bivariados(ds_aux.property_surface_total, ds_aux.property_surface_covered)

<font color=red>verificar la recta elegida para eliminar el grupito de outliers

In [ ]:
analisis_mahalanobis(['property_surface_total', 'property_surface_covered'], 9.9)

#### property_surface_covered vs property_rooms

In [ ]:
mostrar_atipicos_bivariados(ds_filtrado.property_surface_covered, ds_filtrado.property_rooms)

In [ ]:
analisis_mahalanobis(['property_surface_covered', 'property_rooms'], 7.9)

En este caso, vemos del grafico de distribución de distancias mahalanobis, que un umbral razonable seria 7.9, dado que a partir de ahí las distancias grandes son anomalías. También podemos observar que solo 1750 casos salieron como outliers, del total de los datos, lo cual es bastante razonable, siendo esta una proporcion baja. 

En el gráfico no parece que sean pocos, pero esto es debido a que la mayoría de los no outliers, se concentran en la esquina inferior izquierda, llevando el valor medio a esa área.

In [ ]:
cond_property_rooms = ds_filtrado['property_rooms'] > 13
cond_surface_covered = ds_filtrado['property_surface_covered'] > 650

reg_combinacion = ds_filtrado[cond_property_rooms | cond_surface_covered]
reg_combinacion.shape

ds_filtrado.drop(reg_combinacion.index, inplace=True)

In [ ]:
analisis_mahalanobis(['property_surface_covered', 'property_rooms'], 7)

En este caso, vimos conveniente eliminar aquellos registros que tienen valores de property_rooms y property_surface_covered poco probables. Por ejemplo, tener 700 m^2 contra 10 cuartos. 

####property_price vs property_rooms (creemos que vuela)

In [ ]:
#mostrar_atipicos_bivariados(ds_filtrado.property_price, ds_filtrado.property_rooms)

In [ ]:
#analisis_mahalanobis(['property_price', 'property_rooms'], 8)

En este caso, vemos del grafico de distribución de distancias mahalanobis, que un umbral razonable seria 8, dado que a partir de ahí las distancias grandes son anomalías. También podemos observar que solo 1436 casos salieron como outliers, del total de los datos, lo cual es bastante razonable, siendo esta una proporcion baja. En el gráfico no parece que sean pocos, pero esto es debido a que la mayoría de los no outliers, se concentran en la esquina inferior izquierda, llevando el valor medio a esa área.

####property_price vs property_surface_total

In [ ]:
mostrar_atipicos_bivariados(ds_filtrado.property_price, ds_filtrado.property_surface_total)

In [ ]:
analisis_mahalanobis(['property_price', 'property_surface_total'], 6)

En este caso, vemos del grafico de distribución de distancias mahalanobis, que un umbral razonable seria 6, dado que a partir de ahí las distancias grandes son anomalías. También podemos observar que solo 2563 casos salieron como outliers, del total de los datos, lo cual es bastante razonable, siendo esta una proporcion baja. 

En el gráfico no parece que sean pocos, pero esto es debido a que la mayoría de los no outliers, se concentran en la esquina inferior izquierda, llevando el valor medio a esa área.

En este caso, vimos conveniente eliminar aquellos registros que tienen valores de property_price y property_surface_total muy poco probables.

In [ ]:
cond_property_price = ds_filtrado['property_price'] > 4000000
cond_surface_total = ds_filtrado['property_surface_total'] > 750

reg_combinacion = ds_filtrado[cond_property_price | cond_surface_total]
reg_combinacion.shape

ds_filtrado.drop(reg_combinacion.index, inplace=True)

In [ ]:
analisis_mahalanobis(['property_price', 'property_surface_total'], 6, 1)

##Reducción de la dimensionalidad

In [ ]:
ds_aux = ds_filtrado.drop(columns = ['target','latitud','longitud','place_l2', 'id', 'place_l3', 'operation', 'property_type', 'property_title', 'end_date', 'created_on', 'start_date', 'property_currency'], inplace=False)
columnas_elegidas = ds_aux.columns.to_list()


x = ds_aux.loc[:, columnas_elegidas]

x = StandardScaler().fit_transform(x)
# print(x)

from sklearn.decomposition import PCA

pca = PCA(n_components=2) #Luego veremos como determinar la cantidad de componentes
x_transformada = pca.fit_transform(x)

print('Shape before PCA: ', x.shape)
print('Shape after PCA: ', x_transformada.shape)

pca_ds_aux = pd.DataFrame(data = x_transformada
             , columns = ['cp_1', 'cp_2'])

print(pca_ds_aux)



In [ ]:
varianza_explicada = pca.explained_variance_ratio_
varianza_explicada

In [ ]:
varianza_explicada.cumsum()

In [ ]:
pca.get_covariance()


In [ ]:
pca.fit(ds_aux)
ipca_transform_2 = pca.transform(ds_aux)
ipca_transform_2.shape

In [ ]:
# image_recon_2 = pca.inverse_transform(ipca_transform_2)
# plt.figure(figsize=[8, 6])
# # plt.imshow(image_recon_2, cmap=plt)
# plt.show()

In [ ]:
pca_ds_aux['target'] = ds_filtrado.target.values
pca_ds_aux

In [ ]:
sns.set()
 
sns.lmplot(
    x='cp_1', 
    y='cp_2', 
    data=pca_ds_aux, 
    hue='target', 
    fit_reg=False, 
    legend=True
    )
 
plt.title('2D PCA Graph')
plt.show()

<font color=red>ANALIZAR TEORICAMENTE
No olvidar lo de la regla del codo
HACERLO CON TEST Y TRAIN


#Agrupamiento



In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.cluster import KMeans


ds_aux = ds_filtrado.drop(columns = ['target','latitud','longitud','place_l2', 'id', 'place_l3', 'operation', 'property_type', 'property_title', 'end_date', 'created_on', 'start_date', 'property_currency'], inplace=False)


kmeans = KMeans(n_clusters=4, random_state=2)  #Creamos un objeto de la clase KMeans para realizar el agrupamiento

kmeans.fit(ds_aux) #Entrenamos el algoritmo para que aprenda cómo agrupar

y_kmeans = kmeans.predict(ds_aux)

centers = kmeans.cluster_centers_ #Visualicemos los grupos y el centroide de cada uno

In [ ]:
sse = []
list_k = list(range(1, 15))

for k in list_k:
    km = KMeans(n_clusters=k)
    km.fit(ds_aux)
    sse.append(km.inertia_)

# Grafico el SSE por K
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse, '-o')
plt.xlabel='Cantidad de clusters'
plt.ylabel='SSE'
plt.show()

<font color=red>que onda esos valores tan altos en el eje y? 
explicar que nos quedamos con 4

Analizamos con Silhouette

In [ ]:
from sklearn.metrics import silhouette_score

#grafico el indice de siluette variando la cantidad de clusters de 2 a 5

list_k = list(range(2, 6))

for n_clusters in list_k:
    clusterer = KMeans(n_clusters=n_clusters,random_state=2)
    preds = clusterer.fit_predict(ds_aux)

    score = silhouette_score (ds_aux, preds)
    print ("For n_clusters =" + str(n_clusters) + " silhouette score is " + str(score))

<font color=red>cuanto mayor, y mas parecido a 1 es mejor asi que bokita

In [ ]:
#ACLARAR QUE ELGIMOS 2

kmeans = KMeans(n_clusters=2, random_state=2)  #Creamos un objeto de la clase KMeans para realizar el agrupamiento
kmeans.fit(ds_aux) #Entrenamos el algoritmo para que aprenda cómo agrupar
y_kmeans = kmeans.predict(ds_aux)
centers = kmeans.cluster_centers_ #Visualicemos los grupos y el centroide de cada uno


In [ ]:


# def graficar_k_means_2d(var1, var2, ds_a_utilizar, clusters):
#   plt.style.use('default')
#   x = ds_a_utilizar[var1].values
#   y = ds_a_utilizar[var2].values

#   plt.scatter(x,y,c=y_kmeans, alpha=0.8)
#   cs_x=clusters[0]
#   cs_y=clusters[1]

#   plt.scatter(cs_x, cs_y, marker='*', s=100, c='r')  
#   plt.title('KMeans')
#   plt.show()

In [ ]:
# print(ds_aux)
# graficar_k_means('property_price', 'property_bedrooms', ds_aux, kmeans.cluster_centers_)
# print(kmeans.cluster_centers_)